# Django - Prezentacja danych
*[Mikołaj Leszczuk](mailto:mikolaj.leszczuk@agh.edu.pl), [Agnieszka Rudnicka](mailto:rudnicka@agh.edu.pl)*

* Szablony HTML w Django - szablon bazowy dla naszej aplikacji
* Prezentacja danych z bazy
  * Widok listy filmów
  * Rejestracja podstrony listy filmów
  * Szablon HTML listy
* Dalsze prace

## Szablony HTML w Django - szablon bazowy dla naszej aplikacji

Każda strona WWW ma kilka stałych elementów interfejsu, takich jak: menu, stopka, obszar gdzie wyświetlane są treści, czasem pasek boczny i inne. Każda podstrona naszej aplikacji musi więc składać się z tego samego kodu, kopiowanego wszędzie i trudnego w utrzymaniu. Gdy zrobimy gdzieś literówkę, to musimy poprawiać we wszystkich plikach z osobna! Gdy chcemy dodać coś nowego do interfejsu też.

Na szczęście istnieją frameworki! Django jak wiele innych narzędzi ma sposób na taką redundancję kodu - [Django HTML Templating Engine](https://docs.djangoproject.com/en/5.0/topics/templates/). W skrócie nazywane dalej szablonami HTML Django.

Do tej pory mieliśmy do czynienia głównie ze standardową składnią HTML.

Django wzbogaca możliwości HTML między innymi o dziedziczenie szablonów. To pozwala nam napisać jeden raz szablon bazowy naszej aplikacji, który będzie dynamicznie załączany do każdej podstrony, która po nim dziedziczy. Brzmi magicznie? Nie szkodzi, zobaczmy jak to wygląda w praktyce.

### Szablon bazowy dla naszej aplikacji

Na dobry początek utwórzmy w katalogu [`movies/templates/`](movies/templates/) nowy plik [`base.html`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/templates/base.html). Będzie to nasz bazowy szablon wykorzystywany przez inne podstrony.

Zacznijmy od czegoś prostego (proponowana początkowa zawartość [`base.html`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/templates/base.html)):

```html
<!DOCTYPE html>
<html lang="pl">
<head>
    <meta charset="UTF-8">
    <title>Moja biblioteka filmów</title>
</head>
<body>
    
</body>
</html>
```

Powyższy HTML wyświetli nam jedynie pustą stronę z tytułem karty "Moja biblioteka filmów". Dodajmy więc przynajmniej jakiś nagłówek (oczywiście w środku elementu `<body> </body>`):

```html
<h1>
    Witamy w filmotece!
</h1>
```

W ten sposób będziemy wiedzieli, że każda strona wykorzystuje ten kod HTML, bo nam się wyświetli nagłówek.

Następnie dodajmy HTMLowy tag Django:

```django
{% block content %}
    Tu będzie treść...
{% endblock %}
```

Ten fragment kodu informuje Django, że w tym miejscu pliku HTML można wstawić jakąś zawartość w szablonach HTML, które rozszerzają ten plik. Kontynuujmy...

_Tag `{% block %}{% endblock %}` może być pusty, ale jeśli coś w nim umieścimy zadziała to jak domyślna wartość._

W szablonie z poprzednich zajęć [`hello.html`](http://localhost:8888/edit/Documents/Courses/Django/movies/templates/hello.html) dodajmy na górze pliku, w pierwszej linijce `extends`.

```django
{% extends "base.html" %}

<h1>Witaj świecie z HTML!</h1>
```

A następnie wejdźmy na [http://127.0.0.1:8000/hello/](http://127.0.0.1:8000/hello/) w przeglądarce. Rezultaty? :)

To teraz zobaczymy co oznaczało "wstawić jakąś zawartość w szablonach HTML, które rozszerzają ten ([`base.html`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/templates/base.html))". Zmieńmy kod na następujący.

```django
{% extends "base.html" %}

{% block content %}
    <h1>Witaj świecie z HTML!</h1>
{% endblock %}
```

I ponownie udajmy się do przeglądarki.

## Prezentacja danych z bazy

W tej chwili mamy działający model na filmy oraz narzędzie, które pozwala na zarządzanie nimi (panel administracyjny). Jednak chcielibyśmy pochwalić się światu zawartością naszej biblioteczki. W tym celu oczywiście nie rozdamy wszystkim haseł dostępu, lub go nie zdejmiemy, to byłoby niebezpieczne!

Spróbujmy zatem wyświetlić listę filmów korzystając z szablonów HTML Django. To one pozwalają nam dynamicznie dodawać treść pobraną z bazy danych (a w międzyczasie zmienianą przez użytkowników w panelu administracyjnym).

### Widok listy filmów

Udajmy się do [`movies/views.py`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/views.py) aby utworzyć widok listy filmów.

Na dobry początek trzeba zaimportować model (mądre IDE jak PyCharm lub VSCode same to zaproponują/zrobią).
Następnie piszemy kolejną funkcję, która przyjmuje zapytanie (`request`) jako argument.

```python
from movies.models import Movie

def list_movies(request):
    movies = Movie.objects.all()
    return render(
        request, 
        template_name="movie_list.html", 
        context={"movies": movies}
    )
```

W tym miejscu wiele się dzieje! Zacznijmy od góry:

* dzięki zaimportowaniu modelu `Movie` będziemy mogli wejść w interakcje z bazą danych za pośrednictwem Django, czyli między innymi:
  * dodawać nowe obiekty typu `Movie`
  * edytować i usuwać istniejące obiekty `Movie`
  * odczytywać a także filtrować istniejące obiekty `Movie`

* `objects` - każdy model w Django ma coś co się nazywa [manager](https://docs.djangoproject.com/en/5.0/topics/db/managers/), nie będziemy wchodzić tutaj w szczegóły, ale jest to interfejs przez który dostarczane są nam operacje na bazie danych. W praktyce, umożliwia to nam tworzenie, edycję, usuwanie i inne zapytania do bazy danych.

* `Movie.objects.all()` użyje menedżera obiektów typu `Movie` i zapyta bazę danych o WSZYSTKIE filmy. Dostaniemy więc listę całej zawartości naszej filmoteki.

* Do wygenerowania odpowiedzi aplikacji zostanie użyty szablon `movie_list.html` (tak, musimy go teraz stworzyć!)

* Lista filmów zostanie dodana do kontekstu HTML

### Rejestracja podstrony listy filmów

Aby widok mógł być wyświetlony trzeba mu przydzielić jakiś adres. Udajmy się więc do [`urls.py`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/goodmovies/urls.py) czyli o URL resolvera naszego projektu. Tutaj należy dodać `path()`, który będzie kierował zapytania przeglądarek użytkowników z konkretnego adresu na stronę powstającej listy filmów.

Przykładowo, do listy `urlpatterns` dodajmy linijkę:

```python
urlpatterns = [
    ...
    
    # http://127.0.0.1:8000/filmy/
    path("filmy/", views.list_movies),  # NOWE
]
```

Teraz link [http://127.0.0.1:8000/filmy/](http://127.0.0.1:8000/filmy/) powinien nas kierować do listy filmów, którą przed chwilą napisaliśmy w [`views.py`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/views.py). Został jeszcze jeden element - szablon HTML, który wyświetli dane.

### Szablon HTML listy

Stwórzmy plik o nazwie [`movie_list.html`](http://localhost:8888/edit/Documents/Teaching/Courses/Django/movies/templates/movie_list.html) (oczywiście musi być umieszczony w katalogu szablonów naszej aplikacji [`movies/templates/`](movies/templates/)).

Na początek możemy sprawdzić, czy wszystko dobrze zaprogramowaliśmy. Spróbujmy więc wypisać zmienną `movies`, którą dodaliśmy do kontekstu szablonu.

```django
{{ movies }}
```

Jeśli w przeglądarce pojawiła się lista obiektów z tajemniczym `QuerySet`, to jesteśmy na dobrej drodze.

Aby wypisać elementy list prostą pętlą można posłużyć się tagiem Django

`{% for %}`.

```django
{% for movie in movies %}
<p>
    Film: "{{ movie }}"
</p>
{% endfor %}
```

Powyższy kawałek kodu prze-iteruje po liście, którą dostarczyliśmy z widoku. Następnie dla każdego elementu wstawi HTML zawarty w środku, czyli w naszym przypadku akapit z napisem `Film: "{{ movie }}"`.

## Dalsze prace

1. Wypisz osobno każde z pól modelu filmu np.: tytuł (`movie.title`), datę publikacji (`movie.published_at`)
1. Wykorzystaj HTML (np.: `<b></b>`, `<i></i>`) i CSS (np.: `<style>p {color: green;}</style>` wewnątrz `<head></head>`) do poprawienia wyglądu listy
1. Przebuduj szablon `movie_list.html` tak aby wykorzystywał (rozszerzał) szablon bazowy `base.html` (np.: `{% extends "base.html" %}{% block content %}{% endblock %}`)
1. Dodaj widok szczegółów pierwszego, pojedynczego filmu (np.: `Movie.objects.all()[0]`)